## LOAD LIBRARIES

In [1]:
import os
import pandas as pd
import re
import json
import openpyxl

import requests
import time
import datetime

/opt/homebrew/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## ІФС (Індекс фінансового стресу) | FSI (Financial Stress Index)

In [13]:
r = requests.get("https://bank.gov.ua/ua/stability/fsi#:~:text=%D0%86%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%20%D1%84%D1%96%D0%BD%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%BE%D0%B3%D0%BE%20%D1%81%D1%82%D1%80%D0%B5%D1%81%D1%83%20(%D0%86%D0%A4%D0%A1)%20%E2%80%93,%D0%BD%D0%B5%20%D0%B2%D0%BA%D0%B0%D0%B7%D1%83%D1%94%20%D0%BD%D0%B0%20%D0%BC%D0%B0%D0%B9%D0%B1%D1%83%D1%82%D0%BD%D1%96%20%D1%80%D0%B8%D0%B7%D0%B8%D0%BA%D0%B8.")
fsi_pd = pd.DataFrame(json.loads(re.search(r'JSON\.parse\(\'(.*?)\'\)', r.text).group(1)))
fsi_pd.columns = ['date', 'fsi']
fsi_pd.to_csv('./raw data/fsi_index.csv', index=False)

## Статистика NPL загалом

https://bank.gov.ua/ua/stability/npl

In [ ]:
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
previous_month_date = (pd.Timestamp(today_date) - pd.DateOffset(months=1)).strftime('%Y-%m-%d')

In [19]:
try:
    npl_total = pd.read_excel(f'https://bank.gov.ua/ua/file/download?file=Data_NPLs%20_{today_date}.xlsx', sheet_name='4_NPL_ratio_Groups')
except:
    npl_total = pd.read_excel(f'https://bank.gov.ua/ua/file/download?file=Data_NPLs%20_{previous_month_date}.xlsx', sheet_name='4_NPL_ratio_Groups')

In [ ]:
npl_total = npl_total.iloc[4:,1:]
npl_total.columns = npl_total.iloc[0]
npl_total = npl_total[1:]
npl_total['Дата'] = pd.to_datetime(npl_total['Дата'])

In [25]:
npl_total.to_csv('./agg data/agg_npl.csv', index=False)

## Значення пруденційних нормативів в цілому по системі

In [131]:
norms_all = pd.DataFrame()
for sn in range(2003, 2025):
    norm_prep = pd.read_excel('https://bank.gov.ua/files/stat/Ratios_Banks_2024-10-01.xlsx', sheet_name=str(sn))
    norm_prep = norm_prep.iloc[2:, 1:]
    norm_prep.iloc[0,0] = 'norm'
    norm_prep.iloc[0,1] = 'norm_name'

    norm_prep.columns = norm_prep.iloc[0]
    norm_prep = norm_prep[1:]

    norm_prep = norm_prep.loc[:, ~norm_prep.columns.isna()]
    norm_prep = norm_prep.replace('…', 0)

    norm_prep = norm_prep.melt(
        id_vars=['norm', 'norm_name'],      
        var_name='date',                    
        value_name='value'                  
    )
    norm_prep = norm_prep.dropna()
    norm_prep['date'] = norm_prep['date'].replace('0', '').astype(str).apply(lambda x: datetime.date(int(sn), int(x.split('.')[1]), int(x.split('.')[0])))
    norms_all = pd.concat([norms_all, norm_prep])
    time.sleep(2)
    print(sn)

2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


In [ ]:
norms_all.to_csv('./agg data/norms.csv', index=False)

https://bank.gov.ua/ua/open-data/api-dev

## 6. Довідник державних цінних паперів

## 7. Кредити, надані депозитними корпораціями

In [42]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [49]:
requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/loan?date=20200501&json')

<Response [204]>

In [44]:
df_all = pd.DataFrame()
for m in periods_list:
    dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/loan?date={m}&json')
    dr = pd.DataFrame(dr.json())
    df_all = pd.concat([df_all, dr])
    print(m)
    time.sleep(1)
df_all.to_csv('./raw data/loans.csv', index=False)

20160901
20161001
20161101
20161201
20170101
20170201
20170301
20170401
20170501
20170601
20170701
20170801
20170901
20171001
20171101
20171201
20180101
20180201
20180301
20180401
20180501
20180601
20180701
20180801
20180901
20181001
20181101
20181201
20190101
20190201
20190301
20190401
20190501
20190601
20190701
20190801
20190901
20191001
20191101
20191201
20200101
20200201
20200301
20200401


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# donwload raw data

loans_downloaded = pd.read_csv('./raw data/loans.csv')

periods_list = list(set(periods_list) - set(loans_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/loan?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, loans_downloaded])
    df_all.to_csv('./raw data/loans.csv', index=False)

In [ ]:
# donwload aggregated data

pd.pivot_table(loans_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/loans_agg.csv')

## 8. Депозити, залучені депозитними корпораціями

In [37]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [47]:
requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/deposit?date=20170801&json')

<Response [204]>

In [39]:
dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/deposit?date={m}&json')
dr = pd.DataFrame(dr.json())
df_all = pd.concat([df_all, dr])
print(m)
time.sleep(1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [38]:
df_all = pd.DataFrame()
for m in periods_list:
    dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/deposit?date={m}&json')
    dr = pd.DataFrame(dr.json())
    df_all = pd.concat([df_all, dr])
    print(m)
    time.sleep(1)
df_all.to_csv('./raw data/deposits.csv', index=False)

20160901
20161001
20161101
20161201
20170101
20170201
20170301
20170401
20170501
20170601
20170701


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# donwload raw data

deposits_downloaded = pd.read_csv('./raw data/deposits.csv')

periods_list = list(set(periods_list) - set(deposits_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/deposit?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, deposits_downloaded])
    df_all.to_csv('./raw data/deposits.csv', index=False)

In [ ]:
# donwload aggregated data

pd.pivot_table(deposits_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/deposits_agg.csv')

## 9. Цінні папери, акції та інші форми участі в капіталі в портфелі депозитних корпорацій (крім Національного банку України), та цінні папери, крім акцій, випущені депозитними корпораціями (крім Національного банку України)

In [33]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [35]:
# donwload raw data

secr_downloaded = pd.read_csv('./raw data/secr.csv')

periods_list = list(set(periods_list) - set(secr_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/securities?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, secr_downloaded])
    df_all.to_csv('./raw data/secr.csv', index=False)

Absent periods - 0


In [36]:
# donwload aggregated data

pd.pivot_table(secr_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/secr_agg.csv')

## 10. Процентні ставки депозитних корпорацій (крім Національного банку України) за новими кредитами та новими депозитами

In [28]:
# generate range of dates for api process

start_date = pd.Timestamp('20160901')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160901', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [30]:
# donwload raw data

dep_rates_downloaded = pd.read_csv('./raw data/dep_rates.csv')

periods_list = list(set(periods_list) - set(dep_rates_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        dr = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/mir?date={m}&json')
        dr = pd.DataFrame(dr.json())
        df_all = pd.concat([df_all, dr])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, dep_rates_downloaded])
    df_all.to_csv('./raw data/dep_rates.csv', index=False)

Absent periods - 0


In [31]:
# donwload aggregated data

pd.pivot_table(dep_rates_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/dept_rates_agg.csv')

## 11. Доходи та витрати банків

In [23]:
# generate range of dates for api process

start_date = pd.Timestamp('20090201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20090201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [25]:
# donwload raw data

rev_downloaded = pd.read_csv('./raw data/banks_rev.csv')

periods_list = list(set(periods_list) - set(rev_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        rev = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksincexp?date={m}&period=m&json')
        df_rev = pd.DataFrame(rev.json())
        df_all = pd.concat([df_all, df_rev])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, rev_downloaded])
    df_all.to_csv('./raw data/banks_rev.csv', index=False)

Absent periods - 0


In [27]:
# donwload aggregated data

pd.pivot_table(rev_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/banks_rev_agg.csv')

## 12. Основні показники діяльності банків України

In [14]:
# generate range of dates for api process

start_date = pd.Timestamp('20160101')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20160101', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [21]:
# donwload raw data

mm_downloaded = pd.read_csv('./raw data/main_metrics.csv')

periods_list = list(set(periods_list) - set(mm_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        mm = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/basindbank?date={m}&period=m&json')
        df_mm = pd.DataFrame(mm.json())
        df_all = pd.concat([df_all, df_mm])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, mm_downloaded])
    df_all.to_csv('./raw data/main_metrics.csv', index=False)

Absent periods - 0


In [22]:
# donwload aggregated data

pd.pivot_table(mm_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/main_metrics_agg.csv')

## 13. Дані фінансової звітності/згруповані балансові залишки банків України

In [2]:
# generate range of dates for api process

start_date = pd.Timestamp('20180201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20180201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [3]:
# donwload raw data

balances_downloaded = pd.read_csv('./raw data/balances.csv')

periods_list = list(set(periods_list) - set(balances_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksfinrep?date={m}&period=m&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, balances_downloaded])
    df_all.to_csv('./raw data/balances.csv', index=False)

Absent periods - 0


In [4]:
# donwload aggregated data

pd.pivot_table(balances_downloaded, index='dt', columns='txt', values='value', aggfunc='sum').reset_index().to_csv('./agg data/balances_agg.csv')

## 19. Макроекономічні показники. (Базовий індекс споживчих цін)

In [2]:
# generate range of dates for api process

start_date = pd.Timestamp('20100201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20100201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [5]:
# donwload raw data

base_cpi_downloaded = pd.read_csv('./raw data/base_cpi.csv')

periods_list = list(set(periods_list) - set(base_cpi_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/inflation?period=m&date={m}&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, base_cpi_downloaded])
    df_all.to_csv('./raw data/base_cpi.csv', index=False)

Absent periods - 0


## 19. Макроекономічні показники. (Економічна активність)

In [13]:
# generate range of dates for api process

start_date = pd.Timestamp('20100201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20100201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [18]:
df_all = pd.DataFrame()
for m in periods_list:
    try:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/economicactivity?period=m&date={m}&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(1)
    except:
        pass
df_all.to_csv('./raw data/economic_activity.csv', index=False)

20100201
20100301
20100401
20100501
20100601
20100701
20100801
20100901
20101001
20101101
20101201
20110101
20110201
20110301
20110401
20110501
20110601
20110701
20110801
20110901
20111001
20111101
20111201
20120101
20120201
20120301
20120401
20120501
20120601
20120701
20120801
20120901
20121001
20121101
20121201
20130101
20130201
20130301
20130401
20130501
20130601
20130701
20130801
20130901
20131001
20131101
20131201
20140101
20140201
20140301
20140401
20140501
20140601
20140701
20140801
20140901
20141001
20141101
20141201
20150101
20150201
20150301
20150401
20150501
20150601
20150701
20150801
20150901
20151001
20151101
20151201
20160101
20160201
20160301
20160401
20160501
20160601
20160701
20160801
20160901
20161001
20161101
20161201
20170101
20170201
20170301
20170401
20170501
20170601
20170701
20170801
20170901
20171001
20171101
20171201
20180101
20180201
20180301
20180401
20180501
20180601
20180701
20180801
20180901
20181001
20181101
20181201
20190101
20190201
20190301
20190401
2

In [ ]:
# donwload raw data

economic_activity_downloaded = pd.read_csv('./raw data/base_cpi.csv')

periods_list = list(set(periods_list) - set(economic_activity_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/economicactivity?period=m&date={m}&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, economic_activity_downloaded])
    df_all.to_csv('./raw data/economic_activity.csv', index=False)

## 19. Макроекономічні показники. (Ринок праці)

In [3]:
# generate range of dates for api process

start_date = pd.Timestamp('20030201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20030201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [4]:
df_all = pd.DataFrame()
for m in periods_list:
    try:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/labormarket?period=m&date={m}&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(1)
    except:
        pass
df_all.to_csv('./raw data/labor_market.csv', index=False)

20030201
20030301
20030401
20030501
20030601
20030701
20030801
20030901
20031001
20031101
20031201
20040101
20040201
20040301
20040401
20040501
20040601
20040701
20040801
20040901
20041001
20041101
20041201
20050101
20050201
20050301
20050401
20050501
20050601
20050701
20050801
20050901
20051001
20051101
20051201
20060101
20060201
20060301
20060401
20060501
20060601
20060701
20060801
20060901
20061001
20061101
20061201
20070101
20070201
20070301
20070401
20070501
20070601
20070701
20070801
20070901
20071001
20071101
20071201
20080101
20080201
20080301
20080401
20080501
20080601
20080701
20080801
20080901
20081001
20081101
20081201
20090101
20090201
20090301
20090401
20090501
20090601
20090701
20090801
20090901
20091001
20091101
20091201
20100101
20100201
20100301
20100401
20100501
20100601
20100701
20100801
20100901
20101001
20101101
20101201
20110101
20110201
20110301
20110401
20110501
20110601
20110701
20110801
20110901
20111001
20111101
20111201
20120101
20120201
20120301
20120401
2

In [ ]:
# donwload raw data

labor_market_downloaded = pd.read_csv('./raw data/base_cpi.csv')

periods_list = list(set(periods_list) - set(labor_market_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/labormarket?period=m&date={m}&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, labor_market_downloaded])
    df_all.to_csv('./raw data/labor_market.csv', index=False)

## 19. Макроекономічні показники. (Державні фінанси)

In [ ]:
# generate range of dates for api process

start_date = pd.Timestamp('20110201')
today = pd.Timestamp.today()

month_diff = (today.year - start_date.year) * 12 + today.month - start_date.month
periods_list = pd.date_range(start='20110201', periods=month_diff, freq='MS').strftime('%Y%m%d').tolist()

In [ ]:
df_all = pd.DataFrame()
for m in periods_list:
    try:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/budget?period=m&date={m}&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(1)
    except:
        pass
df_all.to_csv('./raw data/gov_finance.csv', index=False)

In [ ]:
# donwload raw data

gov_finance_downloaded = pd.read_csv('./raw data/base_cpi.csv')

periods_list = list(set(periods_list) - set(gov_finance_downloaded['dt'].astype(str).drop_duplicates()))

print(f"Absent periods - {len(periods_list)}")

if len(periods_list) >0:
    df_all = pd.DataFrame()
    for m in periods_list:
        balances = requests.get(f'https://bank.gov.ua/NBUStatService/v1/statdirectory/budget?period=m&date={m}&json')
        df_balances = pd.DataFrame(balances.json())
        df_all = pd.concat([df_all, df_balances])
        print(m)
        time.sleep(3)
    df_all = pd.concat([df_all, gov_finance_downloaded])
    df_all.to_csv('./raw data/gov_finance.csv', index=False)